In [ ]:
y = 'Success_Close'
start_from = 1

#This is done for Success Close and mainbaord. We can do the same for Success High & Success High for SME.

In [ ]:
import pandas as pd
import warnings
import time
warnings.filterwarnings("ignore")
main = pd.read_excel("/kaggle/input/main-board-final-v18/ipo_mainline_final_data_v18.xlsx")

In [ ]:
main

In [ ]:
train = main[main['Close Year'] < 2023]
test = main[main['Close Year'] == 2023]

In [ ]:
text_features = ["full_text_content",'answer_of_question_1',	'answer_of_question_2',	'answer_of_question_3','answer_of_question_4','answer_of_question_5','answer_of_question_6','answer_of_question_7','answer_of_question_8','answer_of_question_9','answer_of_question_10','answer_of_question_11','answer_of_question_12','answer_of_question_13','answer_of_question_14','answer_of_question_15','answer_of_question_16','answer_of_question_17','answer_of_question_18','answer_of_question_19','answer_of_question_20','answer_of_question_21','answer_of_question_22','answer_of_question_23','answer_of_question_24','answer_of_question_25']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os
import shutil


output_folder = "probability_outputs"
os.makedirs(output_folder, exist_ok=True)

full_df = main.copy()

start = time.time()

# Preprocessing to handle NaN entries in the text columns
#for idx, feature in enumerate(tqdm(text_features, desc="Processing Features"), 1):
for idx, feature in enumerate(tqdm(text_features[(start_from-1):], desc="Processing Features"), start_from):
    if (time.time() - start) < (11.5*60*60):
        # Replace NaN entries with an empty string (you can modify this if you want a different placeholder)
        full_df[feature].fillna('', inplace=True)

        full_data = pd.DataFrame({'text': full_df[feature], 'labels': full_df[y]})
        full_dataset = Dataset.from_pandas(full_data)
        full_dataset = full_dataset.map(preprocess_function, batched=True)

        model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)

        training_args = TrainingArguments(
            output_dir=f"./results-{feature}",          # Output directory (can be ignored, we don't save models)
            evaluation_strategy="no",                   # No evaluation during training
            learning_rate=2e-5,                         # Learning rate
            per_device_train_batch_size=8,              # Batch size for training
            per_device_eval_batch_size=8,               # Batch size for evaluation
            num_train_epochs=3,                         # Number of epochs
            weight_decay=0.01,                          # Strength of weight decay
            logging_dir=None,                           # No logging
            save_total_limit=0,                         # No saving of checkpoints
            save_steps=0,                               # No saving steps
            load_best_model_at_end=False                # No need to load best model
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=full_dataset,
            tokenizer=tokenizer
        )

        with tqdm(total=training_args.num_train_epochs, desc=f"Training for feature '{feature}'", leave=False) as pbar:
            for epoch in range(training_args.num_train_epochs):
                trainer.train()
                pbar.update(1)  # Update progress bar for each epoch

        predictions = trainer.predict(full_dataset)
        preds_proba = torch.softmax(torch.tensor(predictions.predictions), dim=-1).numpy()

        feature_probabilities_df = pd.DataFrame({
            f'{feature}_class_0_prob': preds_proba[:, 0],  # Probability for class 0
            f'{feature}_class_1_prob': preds_proba[:, 1]   # Probability for class 1
        })

        output_file = os.path.join(output_folder, f'classification_probabilities_{idx}.csv')
        feature_probabilities_df.to_csv(output_file, index=False)
        print(f"Probabilities for feature '{feature}' saved to {output_file}")

        preds_labels = preds_proba.argmax(axis=1)
        true_labels = full_df['Success_Close'].values
        accuracy = accuracy_score(true_labels, preds_labels)
        print(f"Accuracy for feature '{feature}': {accuracy:.4f}")
        #os.rmdir(f"./results-{feature}")
        shutil.rmtree(f"./results-{feature}")

print("All feature probabilities have been processed and saved.")
